In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import mixed_precision

mixed_precision.set_global_policy('mixed_float16')

generator = ImageDataGenerator(rescale = 1./255,
                               validation_split = 0.20)

train_set = generator.flow_from_directory('../input/exml-prelims-task-1/train/train', batch_size = 32, target_size = (200, 200),
                                         color_mode = 'rgb', class_mode = 'binary', subset = 'training')

valid_set = generator.flow_from_directory('../input/exml-prelims-task-1/train/train', batch_size = 32, target_size = (200, 200),
                                          color_mode = 'rgb', class_mode = 'binary', subset = 'validation')

In [ ]:
def create_model():
  model = tf.keras.models.Sequential([tf.keras.layers.Conv2D(512, (3, 3), activation = 'relu', input_shape = (200, 200, 3)),
                                      tf.keras.layers.MaxPool2D(2, 2),
                                      tf.keras.layers.Conv2D(256, (3, 3), activation = 'relu'),
                                      tf.keras.layers.MaxPool2D(2, 2),
                                      tf.keras.layers.Conv2D(512, (3, 3), activation = 'relu'),
                                      tf.keras.layers.MaxPool2D(2, 2),
                                      tf.keras.layers.Flatten(),
                                      tf.keras.layers.Dense(1024, activation = 'relu'),
                                      tf.keras.layers.Dense(1, activation = 'sigmoid', dtype = 'float32')])
  opt = tf.keras.optimizers.RMSprop(learning_rate=1e-4)
  model.compile(optimizer = opt, loss = 'binary_crossentropy', metrics = 'acc')

  return model


In [ ]:
model = create_model()
model.summary()

In [ ]:
history = model.fit(train_set, steps_per_epoch = 8, validation_data = valid_set, validation_steps = 8,
                              epochs = 50, verbose = 1)

In [ ]:
model.save("okok0.1_0.95.h5")
import matplotlib.pyplot as plt

train_acc = history.history['acc']
val_acc = history.history['val_acc']

epochs = range(len(train_acc))

plt.title('Training and Validation Accuracies as functions of Epochs')
plt.plot(epochs, train_acc, 'r', label = 'Training Accuracy')
plt.plot(epochs, val_acc, 'b', label = 'Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

train_loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure()
plt.title('Training and Validation Losses as functions of Epochs')
plt.plot(epochs, train_loss, 'r', label = 'Training Loss')
plt.plot(epochs, val_loss, 'b', label = 'Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

In [ ]:
import cv2
import numpy as np
import os
res = {}
tally = 0
for image in os.listdir("../input/exml-prelims-task-1/test/test/"):
    print(image, tally)
    img = cv2.imread(f"../input/exml-prelims-task-1/test/test/{image}")
    img = cv2.resize(img, (200,200))
    img = np.expand_dims(img, axis = 0)
    res[image.replace(".png", "")] = int(model.predict(img))
    tally += 1
    
print(res)
# image = cv2.imread("../input/exml-prelims-task-1/test/test/AARqHFiREL_609529.png")
# plt.imshow(image)
# image = cv2.resize(image, (200,200))
# image = np.expand_dims(image, axis = 0)
# model.predict(image)


In [ ]:
import pandas as pd
df = pd.DataFrame(res.items(), columns=['id', 'company'])
df.to_csv("submission3.csv", index=False)